In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [3]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

# 데이터 읽기 함수
def load_dataset(file):
    data = np.loadtxt(file)
    print("DATA=", data)

    input_features = data[:, 0:-1]
    print("X=", input_features)

    labels = np.reshape(data[:, -1], (4, 1))
    print("Y=", labels)

    input_features = torch.tensor(input_features, dtype=torch.float)
    labels = torch.tensor(labels, dtype=torch.float)

    return input_features, labels

# 모델 평가 결과 계산을 위해 텐서를 리스트로 변환하는 함수
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

# 데이터셋 로드
x, y = load_dataset("/gdrive/MyDrive/Colab/Week07/train.txt")

# layer 1 가중치 및 편향 초기화
w1 = torch.randn(2, 2, requires_grad=True)  # 입력 2, 은닉층 2
b1 = torch.randn(1, 2, requires_grad=True)  # 은닉층 2

# layer 2 가중치 및 편향 초기화
w2 = torch.randn(2, 1, requires_grad=True)  # 은닉층 2, 출력층 1
b2 = torch.randn(1, 1, requires_grad=True)  # 출력층 1

print("\n[Init]\nw1 = {0}".format(tensor2list(w1)))
print("b1 = {0}".format(tensor2list(b1)))
print("w2 = {0}".format(tensor2list(w2)))
print("b2 = {0}\n".format(tensor2list(b2)))

# Activation 함수 설정
sigmoid = nn.Sigmoid()

# 이진분류 크로스엔트로피 비용 함수 설정
loss_func = torch.nn.BCELoss()

# 옵티마이저 함수 (역전파 알고리즘을 수행할 함수)
optimizer = torch.optim.SGD([w1, b1, w2, b2], lr=0.2)

# 모델 학습
for epoch in range(1001):
    # H(X) 계산: forward 연산
    z1 = x @ w1 + b1  # 은닉층 계산
    a1 = sigmoid(z1)  # 은닉층 활성화 함수 적용

    z2 = a1 @ w2 + b2  # 출력층 계산
    hx = sigmoid(z2)  # 출력층 활성화 함수 적용

    # 비용 계산
    cost = loss_func(hx, y)
    optimizer.zero_grad()  # 기울기 초기화
    cost.backward()  # 역전파 수행
    optimizer.step()  # 가중치 갱신

    # 100 에폭마다 비용 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())

print("\n[Learned]\nw1 = {0}".format(tensor2list(w1)))
print("b1 = {0}".format(tensor2list(b1)))
print("w2 = {0}".format(tensor2list(w2)))
print("b2 = {0}\n".format(tensor2list(b2)))

# 모델 평가
z1 = x @ w1 + b1
a1 = sigmoid(z1)

z2 = a1 @ w2 + b2
hx = sigmoid(z2)

logits = (hx > 0.5).float()
predicts = tensor2list(logits)
golds = tensor2list(y)

print("\nPRED=", predicts)
print("GOLD=", golds)
print("Accuracy : {0:f}".format(accuracy_score(golds, predicts)))


DATA= [[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
X= [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Y= [[0.]
 [1.]
 [1.]
 [0.]]

[Init]
w1 = [[0.8485202789306641, 1.6756283044815063], [1.4517384767532349, -1.5127580165863037]]
b1 = [[0.2923647165298462, 1.4782321453094482]]
w2 = [[-1.2373652458190918], [-2.1678974628448486]]
b2 = [[2.084904432296753]]

0 0.7094601392745972
100 0.6402789950370789
200 0.6104154586791992
300 0.5795302987098694
400 0.5453154444694519
500 0.5043190121650696
600 0.4525006413459778
700 0.39090093970298767
800 0.328274130821228
900 0.27339959144592285
1000 0.22923874855041504

[Learned]
w1 = [[-3.299468755722046, 4.887457370758057], [3.796504020690918, -4.840497970581055]]
b1 = [[1.5476596355438232, 2.723963975906372]]
w2 = [[-3.938793659210205], [-4.2581892013549805]]
b2 = [[5.844116687774658]]


PRED= [[0.0], [1.0], [1.0], [0.0]]
GOLD= [[0.0], [1.0], [1.0], [0.0]]
Accuracy : 1.000000
